In [1]:
! pip install ultralytics clearml

  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (21.1 MB)
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [2]:
from ultralytics import YOLO

In [11]:
model = YOLO('yolov8s-seg.pt')

100%|██████████| 22.8M/22.8M [00:00<00:00, 151MB/s] 


In [7]:
cls= model.model.names
type(cls)

dict

In [8]:
cls[1]

'bicycle'

In [ ]:
!pip install pycocotools

In [9]:
import os
from pycocotools.coco import COCO
import requests
import json

# Define COCO dataset directory and annotation file
dataDir = '/content/dts'
dataType = 'train2017'  # Change to 'val2017' for validation set
annFile = os.path.join(dataDir, 'annotations', 'instances_{}.json'.format(dataType))

# Initialize COCO api for instance annotations
coco = COCO(annFile)

# Define categories for which you want to download images and annotations
categories = cls                                                  #['category1', 'category2', 'category3']  # Add desired categories here

# Create directories to store images and annotations
save_dir = './coco_data'
os.makedirs(save_dir, exist_ok=True)
img_dir = os.path.join(save_dir, 'images')
anno_dir = os.path.join(save_dir, 'annotations')
os.makedirs(img_dir, exist_ok=True)
os.makedirs(anno_dir, exist_ok=True)

# Function to download images and annotations
def download_images_and_annotations(category_name, num_images=100):
    # Get category ID
    catIds = coco.getCatIds(catNms=[category_name])
    imgIds = coco.getImgIds(catIds=catIds)
    if len(imgIds) < num_images:
        print(f"Warning: Number of images available for {category_name} is less than {num_images}")
    imgIds = imgIds[:num_images]  # Select first num_images

    # Download images and annotations
    for img_id in imgIds:
        img = coco.loadImgs(img_id)[0]
        img_url = img['coco_url']
        img_name = img_url.split('/')[-1]
        save_img_path = os.path.join(img_dir, img_name)
        save_anno_path = os.path.join(anno_dir, img_name.replace('.jpg', '.json'))

        # Download image
        if not os.path.exists(save_img_path):
            with open(save_img_path, 'wb') as f:
                f.write(requests.get(img_url).content)
            print(f"Downloaded {save_img_path}")
        else:
            print(f"Image {img_name} already exists. Skipping.")

        # Get segmentation annotations
        annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
        annotations = coco.loadAnns(annIds)
        with open(save_anno_path, 'w') as f:
            json.dump(annotations, f)
        print(f"Saved segmentation annotations for {img_name} to {save_anno_path}")

# Download images and annotations for each category
for category in categories:
    download_images_and_annotations(category)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '/content/dts/annotations/instances_train2017.json'

In [12]:
'instances_{}.json'.format(dataType)

'instances_train2017.json'

In [13]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip #or open the link and save the zip file

--2024-05-15 18:59:11--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.29.100, 3.5.25.66, 16.182.41.65, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.29.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  73.7MB/s    in 4.0s    

2024-05-15 18:59:15 (59.9 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [14]:
! unzip /content/annotations_trainval2017.zip

Archive:  /content/annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  
